In [1]:
pip install tensorflow keras-tuner pandas scikit-learn pillow


   -------------------- ------------------- 1/2 [keras-tuner]
   ---------------------------------------- 2/2 [keras-tuner]

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


🔥 PART 1 — ULTRA TABULAR (.CSV) AUTO HYPERPARAMETER TUNER

✅ Features

This is now a serious structured-data deep learning tuner.

Tuned:

✅ layers (1–6)

✅ neurons (32–1024)

✅ activation (relu / gelu / swish / elu)

✅ dropout

✅ batch norm

✅ L2 regularization

✅ optimizer (Adam / AdamW / RMSprop / Nadam)

✅ learning rate (log scale)

✅ gradient clipping

✅ label smoothing

✅ initializer

✅ batch 

✅ early stopping

✅ LR scheduler

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras_tuner as kt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow import keras
from tensorflow.keras import layers, regularizers


############################################
# LOAD CSV
############################################

def load_csv(path):

    df = pd.read_csv(path)

    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]

    # detect task
    if "float" in str(y.dtype):
        task = "regression"
    else:
        task = "classification"
        le = LabelEncoder()
        y = le.fit_transform(y)

    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2
    )

    return X_train, X_test, y_train, y_test, task, len(np.unique(y)), X.shape[1]


############################################
# MODEL BUILDER
############################################

def build_csv_model(hp, input_dim, task, num_classes):

    inputs = keras.Input(shape=(input_dim,))
    x = inputs

    ##################################
    # REGULARIZATION
    ##################################

    l2 = hp.Float("l2", 1e-6, 1e-2, sampling="log")

    for i in range(hp.Int("num_layers", 1, 6)):

        x = layers.Dense(
            units=hp.Int(f"units_{i}", 32, 1024, step=32),
            activation=hp.Choice(
                "activation",
                ["relu", "gelu", "swish", "elu"]
            ),
            kernel_initializer=hp.Choice(
                "initializer",
                ["he_normal", "glorot_uniform"]
            ),
            kernel_regularizer=regularizers.l2(l2)
        )(x)

        if hp.Boolean("batch_norm"):
            x = layers.BatchNormalization()(x)

        x = layers.Dropout(
            hp.Float("dropout", 0.0, 0.5, step=0.1)
        )(x)

    ##################################
    # OUTPUT
    ##################################

    if task == "regression":

        outputs = layers.Dense(1)(x)
        loss = "mse"
        metrics = ["mae"]

    else:

        if num_classes == 2:

            outputs = layers.Dense(1, activation="sigmoid")(x)

            loss = keras.losses.BinaryCrossentropy(
                label_smoothing=hp.Float("label_smoothing", 0, 0.1)
            )

        else:

            outputs = layers.Dense(num_classes,
                                   activation="softmax")(x)

            loss = keras.losses.SparseCategoricalCrossentropy(
                label_smoothing=hp.Float("label_smoothing", 0, 0.1)
            )

        metrics = ["accuracy"]

    ##################################
    # OPTIMIZER
    ##################################

    lr = hp.Float("lr", 1e-5, 1e-2, sampling="log")

    optimizer = hp.Choice("optimizer", [
        keras.optimizers.Adam(
            learning_rate=lr,
            clipnorm=hp.Float("clipnorm", 1.0, 5.0)
        ),
        keras.optimizers.AdamW(learning_rate=lr),
        keras.optimizers.RMSprop(learning_rate=lr),
        keras.optimizers.Nadam(learning_rate=lr)
    ])

    model = keras.Model(inputs, outputs)

    model.compile(
        optimizer=optimizer,
        loss=loss,
        metrics=metrics
    )

    return model


############################################
# PIPELINE
############################################

def run_csv_pipeline(csv_path):

    X_train, X_test, y_train, y_test, task, num_classes, input_dim = load_csv(csv_path)

    tuner = kt.Hyperband(
        lambda hp: build_csv_model(hp, input_dim, task, num_classes),
        objective="val_loss",
        max_epochs=40,
        factor=3,
        overwrite=True
    )

    early = keras.callbacks.EarlyStopping(
        patience=8,
        restore_best_weights=True
    )

    reduce_lr = keras.callbacks.ReduceLROnPlateau(
        patience=4,
        factor=0.3
    )

    tuner.search(
        X_train,
        y_train,
        validation_split=0.2,
        epochs=80,
        callbacks=[early, reduce_lr],
        batch_size=kt.HyperParameters().Choice(
            "batch_size",
            [16, 32, 64, 128, 256]
        )
    )

    best_hp = tuner.get_best_hyperparameters(1)[0]

    model = tuner.hypermodel.build(best_hp)

    model.fit(
        X_train,
        y_train,
        validation_split=0.2,
        epochs=80,
        callbacks=[early, reduce_lr],
        batch_size=best_hp.get("batch_size")
    )

    model.evaluate(X_test, y_test)

    return model

IMAGE CNN TUNER (KERAS TUNER ONLY)

Tuned Properly:

✅ conv blocks (1–5)

✅ filters (32–512)

✅ kernel size

✅ batch norm

✅ dropout

✅ spatial dropout

✅ GAP vs Flatten

✅ dense head depth

✅ optimizer

✅ LR

✅ label smoothing

✅ augmentation toggle

In [2]:
import tensorflow as tf
import keras_tuner as kt
from tensorflow import keras
from tensorflow.keras import layers


############################################
# LOAD IMAGES
############################################

def load_images(path):

    train = tf.keras.utils.image_dataset_from_directory(
        path,
        validation_split=0.2,
        subset="training",
        seed=123,
        image_size=(128,128),
        batch_size=32
    )

    val = tf.keras.utils.image_dataset_from_directory(
        path,
        validation_split=0.2,
        subset="validation",
        seed=123,
        image_size=(128,128),
        batch_size=32
    )

    return train.prefetch(tf.data.AUTOTUNE), val.prefetch(tf.data.AUTOTUNE), len(train.class_names)


############################################
# MODEL BUILDER
############################################

def build_image_model(hp, num_classes):

    inputs = keras.Input(shape=(128,128,3))
    x = layers.Rescaling(1./255)(inputs)

    if hp.Boolean("augment"):
        x = keras.Sequential([
            layers.RandomFlip("horizontal"),
            layers.RandomRotation(0.1),
            layers.RandomZoom(0.2),
        ])(x)

    ##################################
    # CONV BLOCKS
    ##################################

    for i in range(hp.Int("conv_blocks", 1, 5)):

        x = layers.Conv2D(
            filters=hp.Int(f"filters_{i}", 32, 512, step=32),
            kernel_size=hp.Choice("kernel_size",[3,5]),
            padding="same",
            activation="relu"
        )(x)

        if hp.Boolean("batch_norm"):
            x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D()(x)

        if hp.Boolean("spatial_dropout"):
            x = layers.SpatialDropout2D(0.2)(x)

    ##################################
    # HEAD
    ##################################

    if hp.Boolean("global_avg_pool"):
        x = layers.GlobalAveragePooling2D()(x)
    else:
        x = layers.Flatten()(x)

    for i in range(hp.Int("dense_layers",1,3)):
        x = layers.Dense(
            hp.Int(f"dense_units_{i}",64,512,64),
            activation="relu"
        )(x)

        x = layers.Dropout(
            hp.Float("dropout",0,0.6,0.1)
        )(x)

    outputs = layers.Dense(
        num_classes,
        activation="softmax"
    )(x)

    lr = hp.Float("lr",1e-5,1e-3,sampling="log")

    optimizer = hp.Choice("optimizer",[
        keras.optimizers.Adam(lr),
        keras.optimizers.AdamW(lr)
    ])

    model = keras.Model(inputs, outputs)

    model.compile(
        optimizer=optimizer,
        loss=keras.losses.SparseCategoricalCrossentropy(
            label_smoothing=hp.Float("label_smoothing",0,0.1)
        ),
        metrics=["accuracy"]
    )

    return model


############################################
# PIPELINE
############################################

def run_image_pipeline(path):

    train, val, num_classes = load_images(path)

    tuner = kt.Hyperband(
        lambda hp: build_image_model(hp, num_classes),
        objective="val_accuracy",
        max_epochs=30,
        overwrite=True
    )

    early = keras.callbacks.EarlyStopping(
        patience=6,
        restore_best_weights=True
    )

    tuner.search(
        train,
        validation_data=val,
        epochs=60,
        callbacks=[early]
    )

    best_hp = tuner.get_best_hyperparameters(1)[0]

    model = tuner.hypermodel.build(best_hp)

    model.fit(
        train,
        validation_data=val,
        epochs=60,
        callbacks=[early]
    )

    model.evaluate(val)

    return model


KeyboardInterrupt: 

In [ ]:
# ✅ Usage
# CSV
from csv_keras_tuner import run_csv_pipeline
model = run_csv_pipeline("data.csv")

# Images
from image_keras_tuner import run_image_pipeline
model = run_image_pipeline("images/")